In [2]:
from Crypto import Random
from Crypto.PublicKey import RSA
import base64

In [3]:
def toBase64(string):
    return base64.b64encode(string)

In [4]:
ori_key = RSA.generate(256*5,Random.new().read)

In [5]:
ori_key

RsaKey(n=13064147292672466434974191692713748332827487299406554828069064397026174626884094771325952983425150029810845419246386278279048760148226361957022302327437043317114895496896447616223807714542169907321897869168189244142008636325323859648653445307512523935052546033903353619030954462029523020158030661751472763401198685235095636933986141339698441862772160234808848495208735912360897646303603, e=65537, d=363341495410812002096413947037759305643917033131606452465458811055020457768080342531219168335101416272117724305061506735005764997637601413830647587589098909514407134186859288850138011357803015642588015467590746186060820598894831751788402644702306132512181506324753718182676776791886169006674213645288238022081116499606714974548821029996842616141050759542638308185212110546652181868405, p=3279421246008769146256566765972363090480481028916463353841687515459061149364694596310159267980101090706370667573760088500086685865714286893913572248488360540237321729096479230442833988824713229, q=3983674

**Testing keys**

In [6]:
from Crypto.Cipher import PKCS1_OAEP

pub_key = PKCS1_OAEP.new(ori_key.public_key())
encrypted = pub_key.encrypt(b'hey whatssup')


In [7]:
str.encode('hey')

b'hey'

In [8]:
prikey = PKCS1_OAEP.new(ori_key)
prikey.decrypt(encrypted)

b'hey whatssup'

**Sharding key**

In [23]:
class Shard(RSA.RsaKey):
    def __init__(self,x,n,d,e,q,p,u):
        RSA.RsaKey.__init__(self,n=n,d=d,e=e,q=q,p=p,u=u)
        self.__x = x 
    def decode(self):
        return super().decode(self)

For 2 shards to be able to form the key, we must have all of the shards in a straight line that follows the rule of y=ax+b.
In here, 'x' and 'b' will randomized; and 'a' will be the hidden values like d,q,p and u. The shards will be given 'x' and 'y'.
2 Shards will be needed to compute 'a', which will be the needed attributes of the private key

In [12]:
#Determining the equation 
import random
b = random.randint(1,1000000)
#Getting a list of 5 random numbers from 1 to 100
li = range(1,1000001)
x = random.sample(li,5)
#Printing equations:
print("The equations are:")
for i in range(1,6):
    print("x{:d}: a*{:d} + {:d}".format(i,x[i-1],b))


The equations are:
x1: a*185088 + 269767
x2: a*489038 + 269767
x3: a*184717 + 269767
x4: a*715980 + 269767
x5: a*541605 + 269767


In [13]:
def sharding_linear(nums,private_key):
    if isinstance(private_key,RSA.RsaKey):
        b = random.randint(1,1000000)
        li = range(1,1000001)
        x = random.sample(li,nums)
        list = []
        for i in range(0,nums):
            n = private_key.n
            d = private_key.d * x[i] + b
            e = private_key.e
            q = private_key.q * x[i] + b 
            p = private_key.p * x[i] + b 
            u = private_key.u * x[i] + b 
            list.append(Shard(x=x[i],n=n,d=d,e=e,q=q,p=p,u=u))
        return list 
    else:
        print("Key is not RSA key")

    
    

In [14]:
shards = sharding_linear(5,ori_key)
shards[0]

RsaKey(n=13064147292672466434974191692713748332827487299406554828069064397026174626884094771325952983425150029810845419246386278279048760148226361957022302327437043317114895496896447616223807714542169907321897869168189244142008636325323859648653445307512523935052546033903353619030954462029523020158030661751472763401198685235095636933986141339698441862772160234808848495208735912360897646303603, e=65537, d=92976181943663504840455557735386304239833217442113038334291186078491294979189615010998735423605771612704748707870799081434095216775481100987952072483349698153460670381343797142439516002370929278813132454032731223043475504692396286628637929558162520860807150016453878453201525766829335559455877878120097803946381063317361508267246911007952050729565760960884801958130297392003920122670327567, p=839177661483675954373885382878199935949231251451491640541257101705850075633230429638999275401964028303034602866784616566464182219549360301857331829810183555362408731901956263236477675068333518030575,

In [17]:
shards[2]

RsaKey(n=13064147292672466434974191692713748332827487299406554828069064397026174626884094771325952983425150029810845419246386278279048760148226361957022302327437043317114895496896447616223807714542169907321897869168189244142008636325323859648653445307512523935052546033903353619030954462029523020158030661751472763401198685235095636933986141339698441862772160234808848495208735912360897646303603, e=65537, d=206301667679304946670322874988569356151559652241794827645362858328930065716138337685800931588987233145145722683170872909068923308008653706758903393757214469833185226719956835616219861468846974251705049302343349776983473327846496520347937137635522398979091537476131913646942047094665047900299551765658208666557437137311696695399075092621907269018727210760714605004481584247283642343062110257, p=186202258927131903355301604405144803914391232340847872767777175440250032999777994483894533076642159829217020134170524064946421936769391495549518718696920623114134890456368994225313671051478392472921

In [18]:
type(shards[0].x)

AttributeError: 'Shard' object has no attribute 'x'

**Combining shards to find key**

In [26]:
def combine_linear_shards(shard1,shard2):
    #Solving linear equation
    d_difference = shard1.d-shard2.d
    q_difference = shard1.q - shard2.q
    p_difference = shard1.p - shard2.p
    u_difference = shard1.u - shard2.u
    x_difference = shard1._Shard__x-shard2._Shard__x
    d = d_difference // x_difference
    q = q_difference // x_difference
    p = p_difference // x_difference
    u = u_difference // x_difference
    
    return RSA.RsaKey(d=d,n=shard1.n,e=shard1.e,q=q,p=p,u=u)
    
    

In [27]:
combined = combine_linear_shards(shards[0],shards[2])

**Testing shards and keys**

In [28]:
#Try out shards
shard1 = PKCS1_OAEP.new(shards[0])
shard1.decrypt(encrypted)

ValueError: Fault detected in RSA decryption

In [29]:
#Try out combined key
combined_key = PKCS1_OAEP.new(combined)
combined_key.decrypt(encrypted)

b'hey whatssup'